In [11]:
from pathlib import Path

# P = Path("D:/Dokumente/")
P = Path("D:/Bilder/")


In [12]:
for _ in P.rglob("*"):
    pass

In [13]:
def elem_type(p):
    if p.is_file():
        return "file"
    elif p.is_dir():
        return "dir"
    else:
        return "other"

def _analyse_folder(p):
    for elem in p.iterdir():
        try:
            t = elem_type(elem)
        except OSError:
            continue

        if t == "file":
            pass
        elif t == "dir":
            _analyse_folder(elem)

In [14]:
_analyse_folder(P)